<a href="https://colab.research.google.com/github/amitkbiswas0/ocr-cnn/blob/master/ocr_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os, glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.utils import np_utils, plot_model
from keras.models import Model
from keras.layers import Input, Dropout, Dense, Flatten, BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.layers.merge import concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
np.random.seed(0)
# data plotting
get_ipython().run_line_magic('matplotlib', 'inline')
sns.set(style='white', context='notebook', palette='deep')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [0]:
# change directory to dataset folder
path = '/content/drive/My Drive/Colab Notebooks/data/'
os.chdir(path)

# concat all the csv files in one dataframe
extension_name = "csv"
all_files = [i for i in glob.glob("*.{}".format(extension_name))]
dataframe = pd.concat([pd.read_csv(f) for f in all_files])
print(dataframe.shape)

(366487, 785)


In [0]:
# separate dataframe into label and features
label = dataframe[["label"]].copy()
dataframe.pop("label")
features = dataframe
print(features.shape, label.shape)

(366487, 784) (366487, 1)


In [0]:
# separate label and features into train and test set
x_train, x_test, y_train, y_test = train_test_split(features, label, test_size=0.25, shuffle=True)
print(f'Train size: {x_train.shape}')
print(f'Test size: {x_test.shape}')

Train size: (274865, 784)
Test size: (91622, 784)


In [0]:
# fit and transform train data between 0-1
# for easier computation using min max scaler
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [0]:
# normalize data to avoid anomolies
# one hot encoding
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
print(y_train.shape, y_test.shape)
print(x_train.shape, x_test.shape)

(274865, 122) (91622, 122)
(274865, 784) (91622, 784)


In [0]:
# change into 28*28 matrix
x_train = np.reshape(x_train, (-1,28,28,1))
x_test = np.reshape(x_test, (-1,28,28,1))
print(x_train.shape)
print(x_test.shape)

(274865, 28, 28, 1)
(91622, 28, 28, 1)


In [0]:
# input layer
visible = Input(shape=(28,28,1))

# first 2 conv2d
layer_1 = Conv2D(32, kernel_size=3, padding="same", activation='relu')(visible)
layer_2 = Conv2D(32, kernel_size=5, padding="same", activation="relu")(layer_1)
layer_3 = MaxPooling2D(pool_size=(2,2))(layer_2)
layer_4 = Dropout(rate=0.25)(layer_3)

# parallel part
# top layers
layer_5 = Conv2D(64, kernel_size=5, padding="same", activation="relu")(layer_4)
layer_6 = BatchNormalization()(layer_5)
layer_7 = Conv2D(64, kernel_size=5, padding="same", activation="relu")(layer_6)
layer_8 = BatchNormalization()(layer_7)
layer_9 = MaxPooling2D(pool_size=(2,2))(layer_8)
layer_10 = Dropout(rate=0.25)(layer_9)
# bottom layers
layer_11 = Conv2D(64, kernel_size=3, padding="same", activation="relu")(layer_4)
layer_12 = Conv2D(64, kernel_size=3, padding="same", activation="relu")(layer_11)
layer_13 = MaxPooling2D(pool_size=(2,2))(layer_12)
layer_14 = Dropout(rate=0.25)(layer_13)

# last part
layer_15 = concatenate([layer_10, layer_14])
layer_16 = Conv2D(64, kernel_size=3, padding="same", activation="relu")(layer_15)
layer_17 = MaxPooling2D(pool_size=(2,2))(layer_16)
layer_18 = Dropout(rate=0.25)(layer_17)

# rest
flatten = Flatten()(layer_18)
hidden = Dense(2048, activation="relu")(flatten)
layer_19 = Dropout(rate=0.25)(hidden)
output = Dense(122, activation="softmax")(layer_19)
model = Model(inputs=visible, outputs=output)

print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 28, 28, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 28, 28, 32)   25632       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 14, 14, 32)   0           conv2d_2[0][0]                   
____________________________________________________________________________________________

In [0]:
optimizer = Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=128)
model.save('ocr_cnn.h5')

Train on 274865 samples, validate on 91622 samples
Epoch 1/10
274865/274865 [==============================] - 3615s 13ms/step - loss: 0.8100 - accuracy: 0.7907 - val_loss: 0.2974 - val_accuracy: 0.9211
Epoch 2/10
274865/274865 [==============================] - 3608s 13ms/step - loss: 0.3581 - accuracy: 0.9045 - val_loss: 0.2453 - val_accuracy: 0.9363
Epoch 3/10
274865/274865 [==============================] - 3618s 13ms/step - loss: 0.2985 - accuracy: 0.9198 - val_loss: 0.2360 - val_accuracy: 0.9386
Epoch 4/10
274865/274865 [==============================] - 3621s 13ms/step - loss: 0.2627 - accuracy: 0.9293 - val_loss: 0.2015 - val_accuracy: 0.9490
Epoch 5/10
274865/274865 [==============================] - 3615s 13ms/step - loss: 0.2329 - accuracy: 0.9367 - val_loss: 0.2026 - val_accuracy: 0.9488
Epoch 6/10
274865/274865 [==============================] - 3613s 13ms/step - loss: 0.2114 - accuracy: 0.9420 - val_loss: 0.1905 - val_accuracy: 0.9527
Epoch 7/10
274865/274865 [===========